In [19]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import gensim
from gensim.models import Word2Vec
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
tweets = pd.read_csv("/content/sample_data/cleanTweets.csv")

In [21]:
tweets = tweets.drop('Unnamed: 0', axis=1)
tweets.head()

,TweetID,RawText,Depression(0 or 1)
0,106,just had a real good moment i missssssssss him...,0
1,217,is reading manga,0
2,220,,0
3,288,need to send em to my accountant tomorrow odd...,0
4,540,add me on myspace myspacecomlookthunder,0


In [22]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   TweetID             10314 non-null  int64 
 1   RawText             10312 non-null  object
 2   Depression(0 or 1)  10314 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 241.9+ KB


In [23]:
tweets['RawText'] = tweets['RawText'].astype(str)

In [24]:
def tokenize(text):
    return word_tokenize(text)

stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

In [25]:
tweets['RawText'] = tweets['RawText'].apply(tokenize)
tweets['RawText'] = tweets['RawText'].apply(remove_stopwords)

In [26]:
tweets.head()

,TweetID,RawText,Depression(0 or 1)
0,106,"[real, good, moment, missssssssss, much]",0
1,217,"[reading, manga]",0
2,220,[],0
3,288,"[need, send, em, accountant, tomorrow, oddly, ...",0
4,540,"[add, myspace, myspacecomlookthunder]",0


In [27]:
#WordEmbeddings
tweets_tokenizados = tweets['RawText']
model = Word2Vec(sentences=tweets_tokenizados, vector_size=100, window=5, min_count=1, sg=0)
model.save("modelo_word2vec")
model = Word2Vec.load("modelo_word2vec")

def get_tweet_vector(tweet, model):
    tweet_vector = np.zeros(model.vector_size)

    for word in tweet:
        if word in model.wv:
            tweet_vector += model.wv[word]

    num_words = len(tweet)
    if num_words > 0:
        tweet_vector /= num_words

    return tweet_vector

tweet_vectors = [get_tweet_vector(tweet, model) for tweet in tweets_tokenizados]

In [28]:
#Matrix TF-IDF
tfidf_vectorizer = TfidfVectorizer()

tweets_as_strings = [" ".join(tokens) for tokens in tweets['RawText']]

tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_as_strings)

feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_matrix_dense = tfidf_matrix.toarray()


###Algorithm: K-Nearest Neighbors

In [29]:
i = 0
sumF1 = 0
t = 30

while(i != t):
  X_train, X_test, y_train, y_test = train_test_split(tweet_vectors, tweets['Depression(0 or 1)'], test_size=0.2, random_state=42, stratify=tweets['Depression(0 or 1)'])

  smote = SMOTE(sampling_strategy='auto', random_state=42)  # Você pode ajustar a estratégia de amostragem conforme necessário

  X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

  knn = KNeighborsClassifier(n_neighbors=40)

  knn.fit(X_resampled,y_resampled)

  y_pred = knn.predict(X_test)

  f1 = f1_score(y_test, y_pred, average='weighted')

  sumF1 = sumF1 + f1

  i = i + 1


print("F1-score:", sumF1/t)

report = classification_report(y_test, y_pred)
print(report)
print("The result above isn't a avarage")

F1-score: 0.5289971036111006
              precision    recall  f1-score   support

           0       0.96      0.38      0.55      1600
           1       0.31      0.95      0.46       463

    accuracy                           0.51      2063
   macro avg       0.63      0.66      0.51      2063
weighted avg       0.81      0.51      0.53      2063

The result above isn't a avarage


In [30]:
i = 0
sumF1 = 0
t = 30

while(i != t):
  X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_dense, tweets['Depression(0 or 1)'], test_size=0.2, random_state=42, stratify=tweets['Depression(0 or 1)'])

  smote = SMOTE(sampling_strategy='auto', random_state=42)  # Você pode ajustar a estratégia de amostragem conforme necessário

  X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

  knn = KNeighborsClassifier(n_neighbors=40)

  knn.fit(X_resampled,y_resampled)

  y_pred = knn.predict(X_test)

  f1 = f1_score(y_test, y_pred, average='weighted')

  sumF1 = sumF1 + f1

  i = i + 1


print("F1-score:", sumF1/t)

report = classification_report(y_test, y_pred)
print(report)
print("The result above isn't a avarage")

F1-score: 0.806429947653123
              precision    recall  f1-score   support

           0       0.98      0.75      0.85      1600
           1       0.52      0.94      0.67       463

    accuracy                           0.79      2063
   macro avg       0.75      0.84      0.76      2063
weighted avg       0.87      0.79      0.81      2063

The result above isn't a avarage


###Algorithm: Naive Bayes

In [31]:
i = 0
sumF1 = 0
t = 30

while(i != t):
  X_train, X_test, y_train, y_test = train_test_split(tweet_vectors, tweets['Depression(0 or 1)'], test_size=0.2, random_state=42,stratify=tweets['Depression(0 or 1)'])

  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Instancie o SMOTE
  smote = SMOTE(sampling_strategy='auto', random_state=42)  # Você pode ajustar a estratégia de amostragem conforme necessário

  # Ajuste o SMOTE aos seus dados de treinamento
  X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

  naive_bayes = MultinomialNB()

  naive_bayes.fit(X_resampled, y_resampled)

  y_pred = naive_bayes.predict(X_test)

  f1 = f1_score(y_test, y_pred, average='weighted')

  sumF1 = sumF1 + f1
  i = i + 1

print("F1-score:", sumF1/t)
report = classification_report(y_test, y_pred)
print(report)
print("The result above isn't a avarage")

F1-score: 0.6287725110332049
              precision    recall  f1-score   support

           0       0.83      0.61      0.70      1600
           1       0.29      0.56      0.38       463

    accuracy                           0.60      2063
   macro avg       0.56      0.58      0.54      2063
weighted avg       0.71      0.60      0.63      2063

The result above isn't a avarage


In [32]:
i = 0
sumF1 = 0
t = 30

while(i != t):
  X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_dense, tweets['Depression(0 or 1)'], test_size=0.2, random_state=42,stratify=tweets['Depression(0 or 1)'])

  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Instancie o SMOTE
  smote = SMOTE(sampling_strategy='auto', random_state=42)  # Você pode ajustar a estratégia de amostragem conforme necessário

  # Ajuste o SMOTE aos seus dados de treinamento
  X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

  naive_bayes = MultinomialNB()

  naive_bayes.fit(X_resampled, y_resampled)

  y_pred = naive_bayes.predict(X_test)

  f1 = f1_score(y_test, y_pred, average='weighted')

  sumF1 = sumF1 + f1
  i = i + 1

print("F1-score:", sumF1/t)
report = classification_report(y_test, y_pred)
print(report)
print("The result above isn't a avarage")

F1-score: 0.9031475637697709
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      1600
           1       0.72      0.91      0.80       463

    accuracy                           0.90      2063
   macro avg       0.84      0.90      0.87      2063
weighted avg       0.91      0.90      0.90      2063

The result above isn't a avarage


###Algorithm: LinearSVC

In [33]:
t = 30
while(i != t):
  X_train, X_test, y_train, y_test = train_test_split(tweet_vectors, tweets['Depression(0 or 1)'], test_size=0.2, random_state=42, stratify=tweets['Depression(0 or 1)'])

  svc_model = LinearSVC()

  svc_model.fit(X_train, y_train)

  y_pred = svc_model.predict(X_test)

  f1 = f1_score(y_test, y_pred, average='weighted')

  sumF1 = sumF1 + f1

  i = i + 1

print("F1-score:", sumF1/t)

F1-score: 0.9031475637697709


In [34]:
t = 30
while(i != t):
  X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_dense, tweets['Depression(0 or 1)'], test_size=0.2, random_state=42, stratify=tweets['Depression(0 or 1)'])

  svc_model = LinearSVC()

  svc_model.fit(X_train, y_train)

  y_pred = svc_model.predict(X_test)

  f1 = f1_score(y_test, y_pred, average='weighted')

  sumF1 = sumF1 + f1

  i = i + 1

print("F1-score:", sumF1/t)
report = classification_report(y_test, y_pred)
print(report)
print("The result above isn't a avarage")

F1-score: 0.9031475637697709
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      1600
           1       0.72      0.91      0.80       463

    accuracy                           0.90      2063
   macro avg       0.84      0.90      0.87      2063
weighted avg       0.91      0.90      0.90      2063

The result above isn't a avarage
